In [ ]:
%load_ext lab_black

In [ ]:
import datetime as dt

import mplfinance as mpf
import pandas as pd
import numpy as np
import pandera as pa

import shared


logger = shared.get_logger()

In [ ]:
klines_btc_1m_df = shared.read_binance_klines()
klines_btc_1h_df = shared.split_df(df=klines_btc_1m_df, interval_str="1h")

### weekends-pumps-dumps on 2024-03-30

In [ ]:
sundays_list = pd.date_range(start="2023-08-01", end="2024-03-30", freq="W").tolist()
weekeds_dt_ranges = [
    (sunday_dt - dt.timedelta(days=2), sunday_dt + dt.timedelta(days=1))
    for sunday_dt in sundays_list
]
dt_threshold = dt.datetime.fromisoformat("2023-08-01")
for dt_start, dt_end in weekeds_dt_ranges:
    slice_df = klines_btc_1h_df[lambda x: x.index >= dt_threshold][
        lambda x: x.index >= dt_start
    ][lambda x: x.index <= dt_end]
    fig, axlist = mpf.plot(
        slice_df,
        warn_too_much_data=len(slice_df),
        type="candle",
        figsize=(14, 3),
        style=shared.s,
        returnfig=True,
        vlines={
            "vlines": [(dt_start + dt.timedelta(days=1)).date()],
            "linewidths": 0.2,
        },
    )
    max_diff_abs = slice_df.high.max() - slice_df.low.min()
    max_diff_rel = (max_diff_abs / slice_df.low.min()) * 100
    x_label = f"[{dt_start.date()} - {dt_end.date()}] max_diff_rel={max_diff_rel:.2f}"
    axlist[-2].set_xlabel(x_label)
    axlist[-2].xaxis.set_ticklabels([])

### aprils on 2024-04-02

In [ ]:
for year in [2023, 2022, 2021, 2020, 2019, 2018]:
    dt_start = dt.datetime.fromisoformat(f"{year}-04-01")
    dt_end = dt.datetime.fromisoformat(f"{year}-05-01")
    slice_df = klines_btc_1h_df[lambda x: x.index >= dt_start][
        lambda x: x.index <= dt_end
    ]
    fig, axlist = mpf.plot(
        slice_df,
        warn_too_much_data=len(slice_df),
        type="candle",
        figsize=(14, 3),
        style=shared.s,
        returnfig=True,
    )
    max_diff_abs = slice_df.high.max() - slice_df.low.min()
    max_diff_rel = (max_diff_abs / slice_df.low.min()) * 100
    x_label = f"[{dt_start.date()} - {dt_end.date()}] max_diff_rel={max_diff_rel:.2f}"
    axlist[-2].set_xlabel(x_label)
    axlist[-2].xaxis.set_ticklabels([])

### ema-hit

In [ ]:
dt_start = dt.datetime.fromisoformat("2024-03-01")
dt_end = dt.datetime.fromisoformat("2024-03-31")
slice_df = klines_btc_1h_df[lambda x: x.index >= dt_start][lambda x: x.index <= dt_end]
highs_vals = []
highs_alines = []

for _, row in slice_df.iterrows():
    while highs_vals and highs_vals[-1] < row.high:
        highs_vals.pop()
        highs_alines.pop()
    ts = dt.datetime.fromtimestamp(row.open_time / 1000)
    aline = [(ts, row.high + 50), (ts, row.high + 100)]
    highs_vals.append(row.high)
    highs_alines.append(aline)
    assert len(highs_vals) == len(highs_alines)

mpf.plot(
    slice_df,
    warn_too_much_data=len(slice_df),
    alines={"alines": list(highs_alines), "colors": ["#f19d38"]},
    type="candle",
    figsize=(14, 4),
    style=shared.s,
    ema=120,
)

In [ ]:
slice_df = klines_btc_1h_df[
    lambda x: x.index >= dt.datetime.fromisoformat("2024-03-01")
].head(30)
t_indicator = mpf.make_addplot(slice_df[["high", "low"]])
fig, axlist = mpf.plot(
    slice_df,
    type="candle",
    figsize=(14, 4),
    style=shared.s,
    returnfig=True,
    addplot=t_indicator,
)
??mpf.make_addplot